<a href="https://colab.research.google.com/github/ucfilho/marquesgabi_fev_2020/blob/master/02_Mahotas_fracionado_fev_14_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mahotas

In [0]:

!git clone https://github.com/ucfilho/marquesgabi #clonar do Github
%cd marquesgabi

Cloning into 'marquesgabi'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 439 (delta 52), reused 0 (delta 0), pack-reused 343
Receiving objects: 100% (439/439), 9.05 MiB | 26.63 MiB/s, done.
Resolving deltas: 100% (246/246), done.
/content/marquesgabi/marquesgabi


In [0]:
import numpy as np
import cv2
from skimage import data
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from skimage import io, util
import mahotas.features.texture as mht
import mahotas.features
import os 
import zipfile
from skimage import filters
from skimage import exposure
import skimage.feature as sk
from google.colab import files
from numpy import linalg as LA
from scipy.signal import find_peaks
from scipy.signal import peak_prominences
from scipy.signal import peak_widths
import re
import Go2BlackWhite
import Go2Mahotas
%matplotlib inline

In [0]:
Size=20 # tamanho da foto
Sub_Size=int(Size/5) # tamanho do fracionamento
Row_Crop=1/2 # posicao do corte
Crop=int(Size*Row_Crop)

In [0]:
!wget http://ftp.feq.ufu.br/ubirajara/GrBuInd.zip
Transfere='GrBuInd.zip'

--2020-01-30 12:58:04--  http://ftp.feq.ufu.br/ubirajara/GrBuInd.zip
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 497627 (486K) [application/zip]
Saving to: ‘GrBuInd.zip’

GrBuInd.zip         100%[===================>] 485.96K   431KB/s    in 1.1s    

2020-01-30 12:58:06 (431 KB/s) - ‘GrBuInd.zip’ saved [497627/497627]



In [0]:
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()

In [0]:
# print(img_name)

In [0]:
# ww contain all photos in black and white
ww,img_name=Go2BlackWhite.BlackWhite(Transfere,Size)
# plt.axis("off")
# plt.imshow(ww[0], cmap = "gray")

In [0]:
print(np.mean(ww[0]))

134.1025


In [0]:
Num,Row,Col=np.asarray(ww).shape
Idx=[]
New_Colunm=[]

for i in range(Num):
  Media=np.mean(ww[i])
  k=np.where(ww[i].ravel() > Media)
  Idx.append(np.asarray(k).ravel())

Quant=Size*Size
for i in Idx:
  #print(i)
  New_Colunm.append(len(i)/Quant)
print(New_Colunm)

[0.5825, 0.4125, 0.41, 0.4225, 0.4825, 0.53, 0.49, 0.38, 0.405, 0.4125, 0.41, 0.53, 0.5775, 0.4675, 0.4675, 0.5, 0.51, 0.53, 0.485, 0.4775, 0.5275, 0.495, 0.5525, 0.48, 0.5275, 0.4325, 0.4575, 0.5475, 0.56, 0.535, 0.5, 0.59, 0.51, 0.525, 0.44, 0.435, 0.5125, 0.39, 0.565, 0.5075, 0.485, 0.4825, 0.3775, 0.5275, 0.5475, 0.5325, 0.58, 0.4275, 0.41, 0.5175, 0.5225, 0.44, 0.4375, 0.395, 0.535, 0.555, 0.4825, 0.4325, 0.4625, 0.4525, 0.495, 0.4925, 0.5875, 0.3625, 0.375, 0.355, 0.4075, 0.405, 0.465, 0.4575, 0.4725, 0.4825, 0.4375, 0.585, 0.5325, 0.57, 0.4775, 0.4425, 0.4025, 0.45, 0.4625, 0.615, 0.5575, 0.4675, 0.4175, 0.45, 0.4525, 0.4625, 0.525, 0.4625, 0.525, 0.455, 0.3875, 0.6025, 0.33, 0.5625, 0.33, 0.3675, 0.41, 0.5, 0.53, 0.4875, 0.47, 0.54, 0.41, 0.5125, 0.4975, 0.5575, 0.405, 0.6125, 0.5575, 0.5075, 0.4025, 0.4875, 0.4825, 0.4575, 0.43, 0.39, 0.45, 0.41, 0.435, 0.505, 0.5325, 0.46]


In [0]:
print(len(New_Colunm))

124


In [0]:
# Find the list elements starting with specific letter
subs = 'B'
Buraco = [x for x in img_name if re.search(subs, x)] 
subs = 'G'
Grao = [x for x in img_name if re.search(subs, x)] 
subs = 'I'
Indef = [x for x in img_name if re.search(subs, x)] 

In [0]:
# find df texture for a given picture
df=Go2Mahotas.Mahotas(ww[0],Size,Sub_Size,Crop,img_name)
print(df)

         ASM    constrast    correl  ...  dif entropy      IMC1      IMC2
0   0.061632  6944.534722  0.369340  ...     3.236717 -0.791553  0.997921
1   0.048611  4261.979167  0.038173  ...     3.278384 -0.855280  0.999272
2   0.048611  1743.333333  0.182779  ...     3.196888 -0.855280  0.999272
3   0.048611  1469.895833 -0.001624  ...     3.071888 -0.855280  0.999272
4   0.048611  1450.479167  0.065797  ...     3.294110 -0.831492  0.998989
5   0.049479  1437.298611 -0.020173  ...     3.182999 -0.824276  0.998878
6   0.049479  1596.666667 -0.169023  ...     3.280222 -0.773185  0.997900
7   0.051022  1228.430556 -0.019456  ...     3.224666 -0.756854  0.997391
8   0.051022  1076.590278  0.139392  ...     3.266333 -0.726133  0.996354
9   0.050154  1741.881944  0.000208  ...     3.377444 -0.768026  0.997761
10  0.048611  2105.875000 -0.043431  ...     3.335777 -0.787909  0.998133
11  0.050347  2192.527778 -0.081103  ...     3.238555 -0.800205  0.998410
12  0.048611  2261.958333 -0.192695  .

In [0]:

print(len(New_Colunm))

124


In [0]:
j=0
cont=-1
Posicao_Buraco=[]
for i in img_name:
  cont=cont+1
  for k in Buraco:
    if(i==k):
      Posicao_Buraco.append(cont)
j=0
cont=-1
Posicao_Grao=[]
for i in img_name:
  cont=cont+1
  for k in Grao:
    if(i==k):
      Posicao_Grao.append(cont)

j=0
cont=-1
Posicao_Indef=[]
for i in img_name:
  cont=cont+1
  for k in Indef:
    if(i==k):
      Posicao_Indef.append(cont)

# print(Posicao_Grao)
# print(Posicao_Buraco)
# print(Posicao_Indef)


In [0]:
print(np.asarray(New_Colunm)[Posicao_Buraco])

[0.4125 0.41   0.4225 0.4825 0.53   0.49   0.38   0.405  0.4125 0.41
 0.51   0.525  0.44   0.435  0.5125 0.39   0.565  0.5075 0.485  0.4825
 0.3625 0.375  0.355  0.4075 0.405  0.465  0.4575 0.4725 0.4825 0.4375
 0.33   0.5625 0.33   0.3675 0.41   0.5    0.53   0.4875 0.47   0.54  ]


In [0]:
# compara constrast dos tres primeiros buracos
Num=len(Posicao_Buraco)
Matriz_X_Buraco=[]
#Efeito_escuro=[]
#Num=3
for i in range(Num):
  k=Posicao_Buraco[i]
  df=Go2Mahotas.Mahotas(ww[k],Size,Sub_Size,Crop,img_name)
  Matriz_X_Buraco.append(np.array(df['constrast']).ravel())
Contraste_Buraco=pd.DataFrame(Matriz_X_Buraco)

In [0]:
Row,Col=Contraste_Buraco.shape
New=np.asarray(New_Colunm)[Posicao_Buraco]
#Matriz_New = np.hstack(Matriz_X_Buraco,New)
#print(Matriz_New)
Contraste_Buraco=pd.DataFrame(Matriz_X_Buraco)
Contraste_Buraco[Col]=New
print(Contraste_Buraco)

             0            1            2   ...           15           16      17
0    289.777778   253.722222   282.965278  ...    77.819444    62.166667  0.4125
1    280.250000   140.520833    36.500000  ...    46.013889    53.847222  0.4100
2     61.888889    55.472222    46.937500  ...   265.451389   458.125000  0.4225
3    927.027778   723.388889   385.076389  ...   554.312500  1048.965278  0.4825
4   1618.798611  1373.083333   775.562500  ...   608.069444   622.652778  0.5300
5     93.312500    63.951389    11.368056  ...   316.034722   229.923611  0.4900
6   1649.951389   362.715278    60.062500  ...    16.645833    62.284722  0.3800
7    168.173611    54.381944    63.979167  ...  2706.055556  1403.770833  0.4050
8    495.055556   422.173611    70.861111  ...   101.090278   148.416667  0.4125
9   1558.736111   937.083333   311.840278  ...   694.777778  1224.319444  0.4100
10   589.472222   605.000000   339.979167  ...   173.986111   248.027778  0.5100
11   392.013889   244.861111

In [0]:
# compara constrast dos graos
Num=len(Posicao_Grao)
Matriz_X_Grao=[]
for i in range(Num):
  k=Posicao_Grao[i]
  df=Go2Mahotas.Mahotas(ww[k],Size,Sub_Size,Crop,img_name)
  Matriz_X_Grao.append(np.array(df['constrast']).ravel())

Contraste_Grao=pd.DataFrame(Matriz_X_Grao)

In [0]:
Row,Col=Contraste_Buraco.shape
New=np.asarray(New_Colunm)[Posicao_Grao]
Contraste_Grao[Col]=New
print(Contraste_Grao)

              0             1   ...            16      18
0    2743.944444   3114.895833  ...   4459.625000  0.5300
1    2098.861111   1051.875000  ...   1356.854167  0.5775
2    4224.437500   1700.881944  ...   1261.777778  0.4675
3   12058.722222  11564.805556  ...   4913.118056  0.4675
4     633.826389    237.520833  ...   2494.805556  0.5000
5     229.368056    302.333333  ...   1188.798611  0.5100
6     321.513889   1770.805556  ...   1261.361111  0.5300
7      84.125000     49.756944  ...  10276.041667  0.4850
8    1920.173611   1644.618056  ...     87.340278  0.4775
9    2311.541667   1923.861111  ...    989.743056  0.5275
10    185.763889    200.229167  ...   1967.354167  0.3775
11    995.138889    304.868056  ...   4159.201389  0.5275
12    314.597222    290.472222  ...    245.083333  0.5475
13   1894.236111   4636.131944  ...    428.819444  0.5325
14   2018.375000   1670.187500  ...   1757.979167  0.5800
15    540.770833    564.083333  ...   1032.881944  0.4275
16    216.4305

In [0]:
# compara constrast dos indefinidos
Num=len(Posicao_Indef)
Matriz_X_Indef=[]
for i in range(Num):
  k=Posicao_Indef[i]
  df=Go2Mahotas.Mahotas(ww[k],Size,Sub_Size,Crop,img_name)
  Matriz_X_Indef.append(np.array(df['constrast']).ravel())

Contraste_Indef=pd.DataFrame(Matriz_X_Indef)

In [0]:
Row,Col=Contraste_Indef.shape
New=np.asarray(New_Colunm)[Posicao_Indef]
Contraste_Indef[Col]=New
print(Contraste_Indef)

             0            1            2   ...           15           16      17
0     20.166667    23.270833    26.368056  ...   206.076389   138.583333  0.4950
1    265.611111   668.256944  1316.118056  ...   705.479167   486.743056  0.5525
2     51.326389    54.388889    58.298611  ...   165.944444   377.611111  0.4800
3    724.527778   327.972222   195.597222  ...    27.222222    25.013889  0.5275
4     54.888889    64.923611   115.534722  ...   136.472222   151.500000  0.4325
5    832.194444   619.506944   403.111111  ...   775.284722   517.159722  0.4575
6     69.902778    48.722222    44.305556  ...   177.326389   112.937500  0.5475
7     60.881944   185.229167   414.500000  ...   109.034722    83.868056  0.5600
8   4104.194444  4771.006944  4508.875000  ...   566.013889   701.979167  0.5350
9    507.201389   460.888889   470.576389  ...    81.923611   319.291667  0.5000
10   331.777778   261.750000   162.104167  ...    48.666667   106.458333  0.4375
11   310.631944   222.840278

In [0]:
print(Contraste_Buraco.head(15))

             0            1           2   ...           15           16      17
0    289.777778   253.722222  282.965278  ...    77.819444    62.166667  0.4125
1    280.250000   140.520833   36.500000  ...    46.013889    53.847222  0.4100
2     61.888889    55.472222   46.937500  ...   265.451389   458.125000  0.4225
3    927.027778   723.388889  385.076389  ...   554.312500  1048.965278  0.4825
4   1618.798611  1373.083333  775.562500  ...   608.069444   622.652778  0.5300
5     93.312500    63.951389   11.368056  ...   316.034722   229.923611  0.4900
6   1649.951389   362.715278   60.062500  ...    16.645833    62.284722  0.3800
7    168.173611    54.381944   63.979167  ...  2706.055556  1403.770833  0.4050
8    495.055556   422.173611   70.861111  ...   101.090278   148.416667  0.4125
9   1558.736111   937.083333  311.840278  ...   694.777778  1224.319444  0.4100
10   589.472222   605.000000  339.979167  ...   173.986111   248.027778  0.5100
11   392.013889   244.861111  164.409722

In [0]:
print(Contraste_Grao.head(15))

              0             1   ...            16      18
0    2743.944444   3114.895833  ...   4459.625000  0.5300
1    2098.861111   1051.875000  ...   1356.854167  0.5775
2    4224.437500   1700.881944  ...   1261.777778  0.4675
3   12058.722222  11564.805556  ...   4913.118056  0.4675
4     633.826389    237.520833  ...   2494.805556  0.5000
5     229.368056    302.333333  ...   1188.798611  0.5100
6     321.513889   1770.805556  ...   1261.361111  0.5300
7      84.125000     49.756944  ...  10276.041667  0.4850
8    1920.173611   1644.618056  ...     87.340278  0.4775
9    2311.541667   1923.861111  ...    989.743056  0.5275
10    185.763889    200.229167  ...   1967.354167  0.3775
11    995.138889    304.868056  ...   4159.201389  0.5275
12    314.597222    290.472222  ...    245.083333  0.5475
13   1894.236111   4636.131944  ...    428.819444  0.5325
14   2018.375000   1670.187500  ...   1757.979167  0.5800

[15 rows x 18 columns]


In [0]:
print(Contraste_Indef.head(15))

             0            1            2   ...          15          16      17
0     20.166667    23.270833    26.368056  ...  206.076389  138.583333  0.4950
1    265.611111   668.256944  1316.118056  ...  705.479167  486.743056  0.5525
2     51.326389    54.388889    58.298611  ...  165.944444  377.611111  0.4800
3    724.527778   327.972222   195.597222  ...   27.222222   25.013889  0.5275
4     54.888889    64.923611   115.534722  ...  136.472222  151.500000  0.4325
5    832.194444   619.506944   403.111111  ...  775.284722  517.159722  0.4575
6     69.902778    48.722222    44.305556  ...  177.326389  112.937500  0.5475
7     60.881944   185.229167   414.500000  ...  109.034722   83.868056  0.5600
8   4104.194444  4771.006944  4508.875000  ...  566.013889  701.979167  0.5350
9    507.201389   460.888889   470.576389  ...   81.923611  319.291667  0.5000
10   331.777778   261.750000   162.104167  ...   48.666667  106.458333  0.4375
11   310.631944   222.840278    61.444444  ...   44.

In [0]:
# found drive
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
# get csv file
Contraste_Buraco.to_csv('Contraste_Buraco_01_jan_2020.csv')

In [0]:
# get csv file
Contraste_Grao.to_csv('Contraste_Grao_01_jan_2020.csv')

In [0]:
# get csv file
Contraste_Indef.to_csv('Contraste_Indef_01_jan_2020.csv')

In [0]:
# save in drive
!cp Contraste_Buraco_01_jan_2020.csv drive/My\ Drive/Maria_Gabriela_Textura_dados_jan_2020/

In [0]:
# save in drive
!cp Contraste_Grao_01_jan_2020.csv drive/My\ Drive/Maria_Gabriela_Textura_dados_jan_2020/

In [0]:
# save in drive
!cp Contraste_Indef_01_jan_2020.csv drive/My\ Drive/Maria_Gabriela_Textura_dados_jan_2020/